<h1>YOLO</h1>

[Details zur Funktionsweise von YOLO folgen]

<h2>1. Install</h2>

Um nicht alles per Hand zu programmieren, gibt es Libraries, die eine bequeme Nutzung von YOLO erlauben.

Als Erstes muss PyTorch installiert werden. <br>
> https://pytorch.org/get-started/locally/ [Letzter Zugriff: 08.07.2024]

Folge diesen Anweisungen, um Ultralytics zu installieren: <br>
> https://github.com/ultralytics/ultralytics [Letzter Zugriff: 08.07.2024] <br>
> https://docs.ultralytics.com/quickstart/   [Letzter Zugriff: 08.07.2024]


Danach kann dieser Befehl in die Konsole eingegeben werden: <br>
`yolo predict model=yolov8s.pt source='https://ultralytics.com/images/bus.jpg'` <br>

Damit wird ein Bild heruntergeladen und direkt für die Predicion genutzt. Dabei wird ein Pre-Trained Model verwendet.

<u>Hinweis</u>:<br>
Um diese Prediction auszuführen, müssen alle Libraries verfügbar sein. <br>
Nicht vorhandene Libraries können z. B. mit `pip install <package>==<version>` installiert werden. <br>
- Bei Numpy kann der folgende Fehler auftreten: `ImportError: numpy.core.multiarray failed to import`. Um das zu beheben muss eine spezifische Numpy Version installiert werden (hier: 1.26.0 statt 2.0.0).
- Dasselbe gilt auf für andere Packages.

<h2>Predict</h2>

In [1]:
# Befehl kann auch direkt im Notebook ausgeführt werden.
!yolo predict model=yolov8s.pt source='https://ultralytics.com/images/bus.jpg'

Ultralytics YOLOv8.2.51 ðŸš€ Python-3.12.4 torch-2.3.1+cpu CPU (Intel Core(TM) i9-9900K 3.60GHz)
YOLOv8s summary (fused): 168 layers, 11156544 parameters, 0 gradients, 28.6 GFLOPs

Found https://ultralytics.com/images/bus.jpg locally at bus.jpg
image 1/1 C:\Users\Alexander\Desktop\ML_DL_self_learn\GitHub\ML_DL_Content\deep_learning\PyTorch\projects\1__YOLO\bus.jpg: 640x480 4 persons, 1 bus, 345.8ms
Speed: 6.0ms preprocess, 345.8ms inference, 11.2ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs\detect\predict3
ðŸ’¡ Learn more at https://docs.ultralytics.com/modes/predict


Das Ergebnis: <br>

Linkes Bild: Iput <br>
Rechtes Bild: Output<br>

<table>
    <tr>
      <td>
      <img src="bus.jpg" hight=300 width=300>
      </td>
      <td>
        <img src="./runs/detect/predict/bus.jpg"  hight=300 width=300>
      </td>
     </tr>
</table>

Wie geht das?: <br>
Das Netzwerk gibt ein Vektor aus, wo unter anderem die Daten für die Boundaries stehen. Diese Müssen nach Klassen in das Bild eingezeichnet werden. <br>
Vor dem Einzeichnen wird ein Algorithmus names <u>IOU</u> verwendet, um überlappenden Boxen zu einer zu verschmelzen.
- Zudem werden noch andere Daten ausgegeben, wie: 4 persons, 1 bus, ..., siehe unten.

Das alles passiert intern, und als Ausgabe bekommen wir das Bild. Die eigentliche Ausgabe des Netzes ist ein Vektor, bestehend aus mehreren sogenannten <u>Anchor Boxen<u>. <br>

Siehe auch: <br>
> YOLOv3 From Scratch Using PyTorch: <br>
> https://www.geeksforgeeks.org/yolov3-from-scratch-using-pytorch/ [Letzter Zugriff: 08.07.2024]


In [1]:
## Als Code: ## 
from ultralytics import YOLO

# Lade Model Typ Nano. 
# - 8n: YOLO Nano, es gibt noch s: small, L: Large und x: extra large.
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Führe Prediction aus. 
results = model("bus.jpg")  


image 1/1 C:\Users\Alexander\Desktop\ML_DL_self_learn\GitHub\ML_DL_Content\deep_learning\PyTorch\projects\1__YOLO\bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 154.0ms
Speed: 3.0ms preprocess, 154.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)


<h2>OpenCV Kamera</h2>

Wir wollen ja nicht nur einzelne Bilder für die Predictions nutzen, sonders auch Videos und Livestreams.

Für diesen Aufbau nutzen wir eine einfache USB Kamera, die über den PC angeschlossen wird. <br>
Dank OpenCV können wir diese ganz einfach nutzen.

Siehe auch: <br>
https://docs.opencv.org/3.4/dd/d43/tutorial_py_video_display.html [Letzter Zugriff: 08.07.2024]

In [2]:
# Teste Quelle.
import cv2 as cv 

def test_device(source):
   cap = cv.VideoCapture(source)  # Source: Standardmäßig ist 0 die Kamera. 
   if cap is None or not cap.isOpened():
       print(f'Warning: Keine Kamera bei der Quelle: {source}')

test_device(0)
test_device(1) 

In [3]:
# Öffne Videokanall. 

vid = cv.VideoCapture(0) 

if not vid.isOpened():
 print("Kamera nicht verfügbar")
 exit()
  
while(True): 
      
    # Erfasse Video:
    ret, frame = vid.read() 
  
    # Öffne Fenster und zeige Frame. 
    cv.imshow('frame', frame) 

    # Durch Taste 'q' soll der Stream beendet werden. 
    if cv.waitKey(1) & 0xFF == ord('q'): 
        break
  
# Freigeben der Kamera. 
vid.release() 
# Schließe alle Fenster. 
cv.destroyAllWindows() 

Jetzt wollen wir ein Video mit der Kamera aufnehmen und speichern. Danach soll das gespeicherte Video geladen und für die Prediction genutzt werden.

Dabei stellen wir ein, dass die Aufnahme n-Sekunden gehen soll.

In [4]:
import time

time_limit  = 15
fps         = 27.0
resolution  = (640, 480)
video_path  = './data/output.mp4'

vid = cv.VideoCapture(0)
if not vid.isOpened():
   print("Kamera nicht verfügbar")
   exit()
    
# Erstelle VideoWriter Objekt. 
fourcc = cv.VideoWriter_fourcc(*'mp4v')  #  mp4v(.mp4, )XVID(.avi), MJPG(.mp4) Weitere Formate auf der OpenCV Webseite. 
out = cv.VideoWriter(video_path, fourcc, fps, resolution)   # VideoWriter(Datei, fourcc, FPS, res( , ) )

# Starte Timer.
start_time = time.time()
while vid.isOpened():
    ret, frame = vid.read()

    if ret:
        out.write(frame)
        # Öffne Fenster und zeige Frame.
        cv.imshow('frame', frame)
        # Limitieren Aufnahmezeit. 
        if time.time() - start_time > time_limit:
            break
        # Drücke Taste zum aufhören. 
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

vid.release()
out.release()
cv.destroyAllWindows()

Wir können testet, ob diese Datei geschrieben wurde.

In [5]:
import os.path
os.path.isfile(video_path)

True

In [6]:
print(f"Größe:    {os.stat(video_path).st_size} bytes")
print(f"Erstellt: {time.ctime(os.stat(video_path).st_ctime)}")
# ggf. weitere Informationen ausgeben... 

Größe:    2421301 bytes
Erstellt: Wed Jul 10 13:29:19 2024


Jetzt kann das Video geladen und abgespielt werden.

In [7]:
cap = cv.VideoCapture(video_path)

while cap.isOpened():
     ret, frame = cap.read()
    
     if not ret:
         print("Keine Frames mehr erhalten")
         break
     # Zeige in einem Fenster das aufgenommene Video.
     cv.imshow('frame', frame)
     if cv.waitKey(1) == ord('q'):
         break
         
cap.release()
cv.destroyAllWindows()

Keine Frames mehr erhalten


In [ ]:
# // Issue with displaying videos, will be fixed soon. 

Alternativ können wir auch Videos hier in dem Notebook einbinden, was ganz nett sein kann. Mit weiteren CSS und HTML Elementen könnte man das weiter ausbauen...
- In Jupyter Notebooks können CSS und HTML Elemente genutzt werden, um Inhalte benutzerdefiniert darzustellen.

<u>Hinweis</u>:<br>
Damit das Video richtig angezeigt wird, muss es ggf. anders codiert werden.: <br>
`ffmpeg -i data/output.mp4 -vcodec h264 -acodec mp2 data/output_enc.mp4`

In [11]:
!ffmpeg -i data/output.mp4 -vcodec h264 -acodec mp2 data/output_enc.mp4

ffmpeg version 4.2.3 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9.3.1 (GCC) 20200523
  configuration: --disable-static --enable-shared --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-amf --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nvenc --enable-nvdec --enable-dxva2 --enable-avisynth --enable-libopenmpt
  libavutil 

In [12]:
from IPython.display import Video

Video("data/output_enc.mp4", embed=True, width=300, height=200)

<h2>Prediction mit gespeicherten Videos</h2>

Mit dem angegebenen Model kann das Video, was wir erstellt haben, jetzt die Inference verwendet werden.

In [18]:
result = model(source="data/output.mp4", show=True, conf=0.5)

# Param:
# - source: Video Datei.
# - show:   Öffne Fenster und zeige Video. 
# - conf:   Ab wie viel % eine Klasse als positive gewertet wird. 
cap.release()
cv.destroyAllWindows()



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/424) C:\Users\Alexander\Desktop\ML_DL_self_learn\GitHub\ML_DL_Content\deep_learning\PyTorch\projects\1__YOLO\data\output.mp4: 480x640 (no detections), 147.5ms
video 1/1 (frame 2/424) C:\Users\Alexander\Desktop\ML_DL_self_learn\GitHub\ML_DL_Content\deep_learning\PyTorch\projects\1__YOLO\data\output.mp4: 480x640 (no detections), 146.1ms
video 1/1 (frame 3/424) C:\Users\Alexander\Desktop\ML_DL_self_learn\GitHub\ML_DL_Content\deep_learning

In [ ]:
# https://www.youtube.com/watch?v=5ku7npMrW40&t=205s
# Kamera kann auch angegeben werden.